# Brute Force Algorithim

In [1]:
import pandas as pd
import numpy as np
from itertools import chain,combinations
import time

In [2]:
def readData(path):
    '''
    Function to read csv file containing the transactions
    
    Parameters:-
    path - Location of the input file
    
    '''
    transactionData = pd.read_csv(path, header = None)
    return transactionData

In [3]:
def frequency(transactionData,support):
    '''Fucntion to determine the frequent items in the transaction database
    
    Parameters:-
    transactionData - Single column dataframe containing all the transactions
    support         - User determined support level for generating itemsets
    
    '''
    
    ## Extract transaction information from dataframe to a list
    Transactions = []
    for i in range(len(transactionData)):
        Transactions.append(transactionData[0][i].split(","))
    
    ## Initialize dictionaries to store frequent itemsets
    FrequentItemSets = {}
    BruteForce = {}
    
    ## Calculate total no. of occurrences of items among all transactions
    for i,items in enumerate(Transactions):
        for j in range(len(items)):
            if items[j] in FrequentItemSets:
                FrequentItemSets[items[j]] += 1
            else:
                FrequentItemSets[items[j]] = 1

    association = []
    nonFrequent = []
    
    ## Assign items to the lists depending on whether they meet the minimum support
    for i in FrequentItemSets:
        if FrequentItemSets[i]/len(Transactions) >= support:
            association.append(i)
        else:
            nonFrequent.append(i)
    
    ## List of all items
    for x in nonFrequent:
        association.append(x)
    
    ## Delete all items which do not meet the minimum support requirement
    for i in nonFrequent:
        del FrequentItemSets[i]

    n_combinations = list(combinations(association,2))
    return n_combinations,Transactions,FrequentItemSets,BruteForce

In [4]:
def support_level(n_combinations,support,Transactions,BruteForce):
    '''
    Function to determine all itemset combinations which meet the minimum support
    
    Parameters:-
    n_combinations - List of all itemset combinations meeting the support requirement
    support        - User determined support level for generating itemsets
    Transaction    - List containing all transactions
    BruteForce - Dictionary containing all itemsets meeting support level
    
    '''
    
    ## Loop to generate itemsets as long as they meet support requirements
    while len(n_combinations) > 0:
        itemSets = []
        for i in n_combinations:
            count = 0
            for j in range(len(Transactions)):
                if set(i).issubset(Transactions[j]) == True: ## for itemsets present in transactions, increase count
                     count += 1
            if count/len(Transactions) >= support:           ## if itemset meets minimum support add it to dictionary
                itemSets.append(i)
                BruteForce[tuple(sorted(i))] = count
        
        itemSets = n_combinations
        
        addition = []
        
        ## Loop to generate (n+1)th itemset
        for i in range(len(itemSets)):
            j = len(itemSets) - 1
            while j > i:                                     ## Generating (n+1)th itemset, eg. (A,B),(A,C) => (A,B,C)
                if len(list(set(itemSets[i]) - set(itemSets[j]))) == 1 and set(itemSets[i]).intersection(set(itemSets[j])) != set():
                    addition.append(tuple(sorted(set(itemSets[i]).intersection(set(itemSets[j])).union(set(itemSets[i]).symmetric_difference(set(itemSets[j]))))))
                j -= 1
        ## List of new itemsets for which support levels need to be checked        
        n_combinations = list(frozenset(sorted(sub)) for sub in set(sorted(addition)))
        
    return BruteForce

In [5]:
def confidence_level(BruteForce,FrequentItemSets,Transactions,confidence,support):
    '''
    Function to generate the support and confidence levels of itemsets which meet user defined requirements
    
    Parameters:-
    BruteForce       -  Dictionary containing frequent itemsets
    FrequentItemSets -  Dictionary containing frequent items
    Transactions     -  List containing all transactions
    confidence       -  User determined confidence level for generating itemsets
    support          -  User determined support level for generating itemsets
    
    '''
    ## iterate over itemsets which are greater than minimum support level
    for i,combination in enumerate(BruteForce):  
    ## iterate to get association of 1 item to the rest of the set
        for j in combination:
            ## for 2-itemsets
            if len(set(combination) - set((j,))) == 1:
                BaseGroup = list(set(combination) - set((j,)),)[0]
                ## calculate support and confidence level
                confidenceCalculation  = BruteForce.get(combination)/FrequentItemSets.get(BaseGroup)
                supportLevel = BruteForce.get(combination)/len(Transactions)
                ## print valid associations
                if confidenceCalculation >= confidence and supportLevel >= support:
                    print(set((BaseGroup,)),"=>","{",j,"}","(",supportLevel*100,"%,",confidenceCalculation*100,"%",")")
            ## for n-itemsets, where n > 2
            else:
                BaseGroup = tuple(set(combination) - set((j,)))
                ## calculate support and confidence level
                confidenceCalculation  = BruteForce.get(combination)/BruteForce.get(tuple(sorted(BaseGroup)))
                supportLevel = BruteForce.get(combination)/len(Transactions)
                ## print valid associations
                if confidenceCalculation >= confidence and supportLevel >= support:
                    print(set(BaseGroup),"=>","{",j,"}","(",supportLevel*100,"%,",confidenceCalculation*100,"%",")")
                
            FrequentItemSets[combination] = BruteForce.get(combination)

In [6]:
def BruteForce(support,confidence,path):
    '''
    Function to execute Brute Force Algorithim
    
    Parameters:-
    support     -  User determined support level for generating itemsets
    confidence  -  User determined confidence level for generating itemsets
    path        -  Location of the input file
    
    '''
    
    transactionData = readData(path)
    n_combinations,Transactions,FrequentItemSets,AprioriResults = frequency(transactionData,support)
    AprioriResults = support_level(n_combinations,support,Transactions,AprioriResults)
    return confidence_level(AprioriResults,FrequentItemSets,Transactions,confidence,support)

## Brute Force Algorithm Execution on Datasets

In [15]:
## Executing Brute Force Algorithm on Dataset 1
start_time = time.time()
BruteForce(.15,.70,"C:/Users/Shank/Desktop/NJIT/CourseMaterial/Spring2022/DataMining/MidTermProject/WorkingDirectory/TransactionDatabase/Database1.csv")
print("Time to execute Brute Force Algorithim --- %s seconds" % (time.time() - start_time))

{'Tomato'} => { Onion } ( 25.0 %, 83.33333333333334 % )
{'Onion'} => { Tomato } ( 25.0 %, 83.33333333333334 % )
{'Milk'} => { Onion } ( 15.0 %, 75.0 % )
{'Shoes'} => { Shirt } ( 25.0 %, 83.33333333333334 % )
{'Shirt'} => { Shoes } ( 25.0 %, 100.0 % )
{'WetWipes'} => { Shoes } ( 15.0 %, 100.0 % )
{'Sunscreen'} => { Shoes } ( 15.0 %, 100.0 % )
{'LightBulb'} => { Heater } ( 20.0 %, 80.0 % )
{'Heater'} => { LightBulb } ( 20.0 %, 80.0 % )
{'HardDisk'} => { Heater } ( 15.0 %, 100.0 % )
{'HardDisk'} => { LightBulb } ( 15.0 %, 100.0 % )
{'Mouse'} => { LightBulb } ( 15.0 %, 75.0 % )
{'Heater', 'LightBulb'} => { HardDisk } ( 15.0 %, 75.0 % )
{'HardDisk', 'LightBulb'} => { Heater } ( 15.0 %, 100.0 % )
{'Heater', 'HardDisk'} => { LightBulb } ( 15.0 %, 100.0 % )
{'Tomato', 'Eggs'} => { Onion } ( 15.0 %, 75.0 % )
{'Onion', 'Eggs'} => { Tomato } ( 15.0 %, 75.0 % )
Time to execute Brute Force Algorithim --- 1629.5015213489532 seconds


In [11]:
## Executing Brute Force Algorithm on Dataset 2
start_time = time.time()
BruteForce(.15,.65,"C:/Users/Shank/Desktop/NJIT/CourseMaterial/Spring2022/DataMining/MidTermProject/WorkingDirectory/TransactionDatabase/Database2.csv")
print("Time to execute Brute Force Algorithim --- %s seconds" % (time.time() - start_time))

{'Eggs'} => { Milk } ( 25.0 %, 83.33333333333334 % )
{'Eggs'} => { Tomato } ( 20.0 %, 66.66666666666666 % )
{'Tomato'} => { Milk } ( 25.0 %, 71.42857142857143 % )
{'Onion'} => { Tomato } ( 20.0 %, 100.0 % )
{'ChickenBreast'} => { Tomato } ( 20.0 %, 80.0 % )
{'Shorts'} => { Shirt } ( 15.0 %, 100.0 % )
{'Shirt'} => { Shorts } ( 15.0 %, 75.0 % )
{'Shoes'} => { Shirt } ( 20.0 %, 100.0 % )
{'Shirt'} => { Shoes } ( 20.0 %, 100.0 % )
{'Shorts'} => { Shoes } ( 15.0 %, 100.0 % )
{'Shoes'} => { Shorts } ( 15.0 %, 75.0 % )
{'Chips'} => { MangoJuice } ( 15.0 %, 75.0 % )
{'WetWipes'} => { Sunscreen } ( 15.0 %, 75.0 % )
{'Sunscreen'} => { WetWipes } ( 15.0 %, 75.0 % )
{'WetWipes'} => { ShavingCream } ( 15.0 %, 75.0 % )
{'ShavingCream'} => { WetWipes } ( 15.0 %, 75.0 % )
{'ShavingCream'} => { Moisturizer } ( 15.0 %, 75.0 % )
{'Moisturizer'} => { ShavingCream } ( 15.0 %, 75.0 % )
{'Chocolates'} => { MangoJuice } ( 15.0 %, 100.0 % )
{'Tomato', 'Eggs'} => { Milk } ( 15.0 %, 75.0 % )
{'ChickenBreast', 'T

In [13]:
## Executing Brute Force Algorithm on Dataset 3
start_time = time.time()
BruteForce(.15,.75,"C:/Users/Shank/Desktop/NJIT/CourseMaterial/Spring2022/DataMining/MidTermProject/WorkingDirectory/TransactionDatabase/Database3.csv")
print("Time to execute Brute Force Algorithim --- %s seconds" % (time.time() - start_time))

{'Milk'} => { Eggs } ( 35.0 %, 87.5 % )
{'Eggs'} => { Milk } ( 35.0 %, 87.5 % )
{'Onion'} => { Eggs } ( 20.0 %, 100.0 % )
{'Onion'} => { Milk } ( 15.0 %, 75.0 % )
{'Onion'} => { Tomato } ( 15.0 %, 75.0 % )
{'Chocolates'} => { Banana } ( 15.0 %, 100.0 % )
{'Mouse'} => { HardDisk } ( 15.0 %, 100.0 % )
{'HardDisk'} => { Mouse } ( 15.0 %, 75.0 % )
{'Mouse'} => { Heater } ( 15.0 %, 100.0 % )
{'Heater'} => { Mouse } ( 15.0 %, 75.0 % )
{'Heater'} => { HardDisk } ( 20.0 %, 100.0 % )
{'HardDisk'} => { Heater } ( 20.0 %, 100.0 % )
{'LightBulb'} => { HardDisk } ( 15.0 %, 100.0 % )
{'HardDisk'} => { LightBulb } ( 15.0 %, 75.0 % )
{'LightBulb'} => { Heater } ( 15.0 %, 100.0 % )
{'Heater'} => { LightBulb } ( 15.0 %, 75.0 % )
{'Heater', 'LightBulb'} => { HardDisk } ( 15.0 %, 100.0 % )
{'HardDisk', 'LightBulb'} => { Heater } ( 15.0 %, 100.0 % )
{'Heater', 'HardDisk'} => { LightBulb } ( 15.0 %, 75.0 % )
{'Tomato', 'Onion'} => { Eggs } ( 15.0 %, 100.0 % )
{'Tomato', 'Eggs'} => { Onion } ( 15.0 %, 75.0 %

In [16]:
## Executing Brute Force Algorithm on Dataset 4
start_time = time.time()
BruteForce(.15,.60,"C:/Users/Shank/Desktop/NJIT/CourseMaterial/Spring2022/DataMining/MidTermProject/WorkingDirectory/TransactionDatabase/Database4.csv")
print("Time to execute Brute Force Algorithim --- %s seconds" % (time.time() - start_time))

{'Lamp'} => { Bed } ( 15.0 %, 75.0 % )
{'Bedsheet'} => { Bed } ( 15.0 %, 75.0 % )
{'Pillow'} => { Bed } ( 15.0 %, 100.0 % )
{'Desk'} => { Chair } ( 15.0 %, 75.0 % )
{'Chair'} => { Desk } ( 15.0 %, 60.0 % )
{'PillowCover'} => { Bedsheet } ( 15.0 %, 100.0 % )
{'Bedsheet'} => { PillowCover } ( 15.0 %, 75.0 % )
{'Pen'} => { Notebook } ( 20.0 %, 66.66666666666666 % )
{'Notebook'} => { Pen } ( 20.0 %, 100.0 % )
{'Ruler'} => { Pen } ( 15.0 %, 75.0 % )
{'StickyNotes'} => { Pen } ( 15.0 %, 75.0 % )
{'StickyNotes'} => { Ruler } ( 15.0 %, 75.0 % )
{'Ruler'} => { StickyNotes } ( 15.0 %, 75.0 % )
{'WaterBottle'} => { Milk } ( 15.0 %, 60.0 % )
{'Milk'} => { WaterBottle } ( 15.0 %, 75.0 % )
{'WaterBottle'} => { Water } ( 15.0 %, 60.0 % )
{'Water'} => { WaterBottle } ( 15.0 %, 100.0 % )
{'WaterBottle'} => { MangoJuice } ( 20.0 %, 80.0 % )
{'MangoJuice'} => { WaterBottle } ( 20.0 %, 80.0 % )
{'Milk'} => { MangoJuice } ( 15.0 %, 75.0 % )
{'MangoJuice'} => { Milk } ( 15.0 %, 60.0 % )
{'MangoJuice'} => { 

In [17]:
## Executing Brute Force Algorithm on Dataset 5
start_time = time.time()
BruteForce(.15,.65,"C:/Users/Shank/Desktop/NJIT/CourseMaterial/Spring2022/DataMining/MidTermProject/WorkingDirectory/TransactionDatabase/Database5.csv")
print("Time to execute Brute Force Algorithim --- %s seconds" % (time.time() - start_time))

{'Milk'} => { Eggs } ( 15.0 %, 75.0 % )
{'Eggs'} => { Milk } ( 15.0 %, 75.0 % )
{'Chocolates'} => { Chips } ( 30.0 %, 85.71428571428571 % )
{'Chips'} => { Chocolates } ( 30.0 %, 100.0 % )
{'Onion'} => { Banana } ( 15.0 %, 100.0 % )
{'Tomato'} => { Banana } ( 15.0 %, 100.0 % )
{'Oil'} => { Water } ( 15.0 %, 100.0 % )
{'ShavingCream'} => { RazerBlades } ( 20.0 %, 80.0 % )
{'RazerBlades'} => { ShavingCream } ( 20.0 %, 80.0 % )
{'RazerBlades'} => { Sunscreen } ( 20.0 %, 80.0 % )
{'WetWipes'} => { RazerBlades } ( 20.0 %, 66.66666666666666 % )
{'RazerBlades'} => { WetWipes } ( 20.0 %, 80.0 % )
{'ShavingCream'} => { Sunscreen } ( 20.0 %, 80.0 % )
{'WetWipes'} => { Sunscreen } ( 25.0 %, 83.33333333333334 % )
{'Moistuerizer'} => { Sunscreen } ( 20.0 %, 100.0 % )
{'Moistuerizer'} => { WetWipes } ( 15.0 %, 75.0 % )
{'WetWipes', 'RazerBlades'} => { Sunscreen } ( 15.0 %, 75.0 % )
{'Sunscreen', 'RazerBlades'} => { WetWipes } ( 15.0 %, 75.0 % )
{'WetWipes', 'Moistuerizer'} => { Sunscreen } ( 15.0 %, 